In [3]:
from scipy.signal import hilbert
import numpy as np
import os 
import matplotlib.pyplot as plt
import pandas as pd
def get_signal_length(signal):
      return len(signal)

In [4]:
def process_file(file_path, save_folder):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {file_path} не найден.")
    data = pd.read_excel(file_path)
    start_time = 0
    filtered_data = data[(data['Time'] >= start_time)]

    r1 = filtered_data.iloc[:, 1]  
    r2 = filtered_data.iloc[:, 2]  
    plt.figure(figsize=(34, 6))
    plt.rc('font', size=40)
    plt.plot(r1, linewidth= 4)
    plt.title("Channel 1")
    plt.ylabel("R, Ω")
    plt.figure(figsize=(34, 6))
    plt.rc('font', size=40)
    plt.plot(r2, linewidth= 4)
    plt.title("Channel 2")
    plt.ylabel("R, Ω")

    a1 = 0.03125
   
    if not r1.empty and 0 in r1.index:
        if r1[0] is not None:
            r11 = [r1[0] * a1]
        else:
            r11 = [0]
    else:
        r11 = [0]  
        
    for i in range(1, len(r1)):
        r11.append(a1 * r1[i] + (1 - a1) * r11[i - 1])

    a2 = 0.03125
    if not r2.empty and 0 in r2.index:
        if r2[0] is not None:
            r22 = [r2[0] * a2]
        else:
            r22 = [0]
    else:
        r22 = [0]  
    
    for i in range(1, len(r2)):
        r22.append(a2 * r2[i] + (1 - a2) * r22[i - 1])
    

    window_size = 80
    step_size = 1

    envelope_full1 = np.zeros_like(r11)
    instantaneous_phase_full1 = np.zeros_like(r11)
    for i in range(0, len(r11) - window_size + 1, step_size):
        window_signal = r11[i:i + window_size]
        analytic_signal = hilbert(window_signal)
        envelope = np.abs(analytic_signal)
        envelope_full1[i:i + window_size] = envelope
        instantaneous_phase1 = np.angle(analytic_signal)
        instantaneous_phase_full1[i:i + window_size] = instantaneous_phase1

    envelope_full2 = np.zeros_like(r22)
    instantaneous_phase_full2 = np.zeros_like(r22)
    for i in range(0, len(r22) - window_size + 1, step_size):
        window_signal2 = r22[i:i + window_size]
        analytic_signal2 = hilbert(window_signal2)
        envelope2 = np.abs(analytic_signal2)
        envelope_full2[i:i + window_size] = envelope2
        instantaneous_phase2 = np.angle(analytic_signal2)
        instantaneous_phase_full2[i:i + window_size] = instantaneous_phase2

    LEN = 61  
    coeff = [0] * LEN 

    for k in range(LEN):
        n = LEN
        coeff[k] = -(12.0 * k - 6 * (n - 1)) / (n ** 3 - n) 
    
    coef1 = instantaneous_phase_full1
    filtered1 = np.convolve(coef1, coeff, mode='same')
    coef2 = instantaneous_phase_full2
    filtered2 = np.convolve(coef2, coeff, mode='same')
    abs_coef1 = np.abs(filtered1)
    abs_coef2 = np.abs(filtered2)
    abs_3 = abs_coef1 + abs_coef2
    noisy = abs_3.median()

    def signum(x):
        return int(np.copysign(1, x))

    rect1 = []
    if noisy < 0.15 and noisy > 0.05:
        for x in instantaneous_phase_full1:
            if abs(x) < 0.75:
                rect1.append(0)
            else:
                rect1.append(signum(x))
    
        rect2 = []
        for x in instantaneous_phase_full2:
            if abs(x) < 0.75:
                rect2.append(0)
            else:
                rect2.append(signum(x))
    else:
        for x in instantaneous_phase_full1:
            if abs(x) < 2.35:
                rect1.append(0)
            else:
                rect1.append(signum(x))
    
        rect2 = []
        for x in instantaneous_phase_full2:
            if abs(x) < 2.35:
                rect2.append(0)
            else:
                rect2.append(signum(x))

    grasp = np.zeros_like(rect1)
    rotation = np.zeros_like(rect1)

    for i in range(len(rect1)):
        if rect1[i] == 1 and rect2[i] == 1:
            rotation[i] = 1
            grasp[i] = 0
        elif rect1[i] == -1 and rect2[i] == -1:
            rotation[i] = -1
            grasp[i] = 0
        elif rect1[i] == 1 and rect2[i] == -1:
            grasp[i] = 1
            rotation[i] = 0
        elif rect1[i] == -1 and rect2[i] == 1:
            grasp[i] = -1
            rotation[i] = 0
        else:
            grasp[i] = 0
            rotation[i] = 0

    filtered_data['grasp'] = grasp
    filtered_data['rotation'] = rotation
    filtered_data['rect1'] = rect1
    filtered_data['rect2'] = rect2
    
    filtered_data['coef1'] = filtered1
    filtered_data['coef2'] = filtered2
    filtered_data['abs_coef1'] = abs_coef1
    filtered_data['abs_coef2'] = abs_coef2
    filtered_data['summ_abs_coef'] = abs_coef1 + abs_coef2
    median_value = filtered_data['summ_abs_coef'].median()
    filtered_data['median_summ_abs_coef'] = None  
    filtered_data.loc[0, 'median_summ_abs_coef'] = median_value  


    output_path = os.path.join(save_folder, os.path.basename(file_path))
    filtered_data.to_excel(output_path, index=False)
    print(f"Файл обработан и сохранен: {output_path}")
    
    plt.figure(figsize=(34, 6))
    plt.rc('font', size=40)
    plt.plot(grasp, linewidth= 4)
    plt.title(f"Grasp {output_path}")
    plt.xlim(200,3000)
    
    plt.figure(figsize=(34, 6))
    plt.rc('font', size=40)
    plt.plot(rotation, linewidth= 4)
    plt.title(f"Rotation {output_path}")
    plt.xlim(200,3000)


def process_all_files(input_folder, save_folder):
    os.makedirs(save_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(input_folder, file_name)
            process_file(file_path, save_folder)

input_folder = 'your_folder'  
save_folder = 'your_folder'

process_all_files(input_folder, save_folder)